In [2]:
import sys, os, pickle
import tweepy as tw
import pandas as pd

In [35]:
def twitter_auth():
    try:
        consumer_key = os.environ['TWITTER_API_KEY']
        consumer_secret = os.environ['TWITTER_API_SECRET']
        access_token = os.environ['TWITTER_API_ACCESS_TOKEN']
        access_secret=os.environ['TWITTER_API_ACCESS_TOKEN_SECRET']
    except KeyError:
        sys.stderr.write("Environment variable not set\n")
        sys.exit(1)
    
    auth = tw.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    
    return auth

In [36]:
def get_twitter_client():
    auth = twitter_auth()
    client = tw.API(auth, wait_on_rate_limit=True)
    return client

In [37]:
# Gather the data

In [38]:
search_terms = "#FED" + " -filter:retweets"
date_since = '2022-06-21'

In [39]:
client = get_twitter_client()
tweets = tw.Cursor(client.search_tweets, q=search_terms, lang='en', since=date_since).items(100)
saved_tweets = {}

for tweet in tweets:
    saved_tweets[tweet.user.screen_name] = tweet.text

# for tweet in tweets:
#     saved_tweets.append(tweet.text)

Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since


In [44]:
for user, tweet in saved_tweets.items():
    print(user + "->" + tweet)
    
    
# print(saved_tweets)

KitcoNewsNOW->#Powell on #China: I don’t expect we’ve seen the full effects of China lockdowns … As long as the zero-COVID policy… https://t.co/uX6szCxHPH


In [45]:
#pickle tweets for later use instead of hitting the API all the time

with open('FED/tweets.txt', 'wb') as file:
    pickle.dump(saved_tweets, file)


In [46]:
# read from file
with open('FED/tweets.txt', 'rb') as file:
    data = pickle.load(file)

In [47]:
data

{'KitcoNewsNOW': '#Powell on #China: I don’t expect we’ve seen the full effects of China lockdowns … As long as the zero-COVID policy… https://t.co/uX6szCxHPH',
 'xglobalmarkets': 'EFFECTIVE FED FUNDS RATE 1.58% JUNE 21 VS 1.58% JUNE 17.\n#XGlobalMarkets #rates #inflation #Fed #economy',
 'KVanderschrick': "#Fed's Powell - There's some evidence of wages flattening out:: We are seeing a slowing in housing.Rate rises shoul… https://t.co/M6PUalKako",
 'pleasantville73': '@financialjuice ‘We don’t think” operative words and probably literal! #FED',
 'aLtErNaTePeTeR': "#pOwElL ReFuSeD To aNsWeR A QuEsTiOn fRoM A SeNaToR As tO WhAt #CoNgReSs cAn dO To mAkE ThE #fEd's jOb oF FiGhTiNg… https://t.co/VVukpGqSiI",
 'VanessaBrownTV': 'My non-expert take on #Powell’s #Fed plan— \n1. Wreck the economy… to fix the economy \n2. Help people lose $ (stocks… https://t.co/mUlqzv3E6W',
 'RealMMOShow': 'Cortez Masto (D): \n\nQ: What can you do about oil companies driving up prices while making record profit

In [48]:
len(data)

65

In [39]:
# Clean Data

In [7]:
# Convert list to one gigantic string

# def combine_tweets(list_of_tweets):
#     combined_tweet = ''.join(list_of_tweets)
#     return combined_tweet

In [8]:
# data_combined =  combine_tweets(data)

In [9]:
# data_combined

'@SeedifyFund @MNZToken #Powell #BNB #Fed #Bitcoin #Binance #BTC Lets go MNZToken 🤑🤑🤑🚀🚀🚀🚀 1000x🚀🚀🚀@SquawkStreet July Scholarship, get a new Mac Pro? #cnbc #Fed Like it or not.She belongs in a hospital\n@SenWarren #FederalReserve #fed #inflation https://t.co/c281q0JmKk🇺🇸 FED\'s Powell:\n\nFinancial conditions have tightened significantly, reflecting both actions taken so far and actio… https://t.co/ROL9hDIuAO🇺🇸 FED\'s Powell:\n\nFinancial conditions have tightened significantly, reflecting both actions taken so far and actio… https://t.co/nlVVNrWg1kThom Tillis (R) opening statements: \n\n\'Morgan Stanley blames inflation on excess government spending.\' \n\n\'Monetary… https://t.co/bF1lWhgjV5Dear God, @SenWarren is a moron. #FedThe Fed: “Everything’s good. Everything’s fine. What’s not fine is Putin’s fault.” \n\n🤦🏻\u200d♂️ #election2022 #Fed #Bidenflation #economy🇺🇸 FED\'s Powell:\n\nThe Fed will continue to communicate its thinking as clearly as possible.\n\n#eurusd #dollar #fed https

In [49]:
# Apply first round of data cleaning

import re
import string

def clean_text_round1(text):
    '''Make text lowercase and remove punctuation'''
    text = text.lower()
    text = re.sub('\[.*?\]','', text)
    text = re.sub('["%s"]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub(r'http\S+', '', text)
    
    return text

# round1 = lambda x: clean_text_round1(x)

data_clean_1 = {}

for user, tweet in data.items():
    data_clean_1[user] = clean_text_round1(tweet)

In [50]:
data_clean_1

{'KitcoNewsNOW': 'powell on china i don’t expect we’ve seen the full effects of china lockdowns … as long as the zerocovid policy… ',
 'xglobalmarkets': 'effective fed funds rate  june  vs  june \nxglobalmarkets rates inflation fed economy',
 'KVanderschrick': 'feds powell  theres some evidence of wages flattening out we are seeing a slowing in housingrate rises shoul… ',
 'pleasantville73': 'financialjuice ‘we don’t think” operative words and probably literal fed',
 'aLtErNaTePeTeR': 'powell refused to answer a question from a senator as to what congress can do to make the feds job of fighting… ',
 'VanessaBrownTV': 'my nonexpert take on powell’s fed plan— \n wreck the economy… to fix the economy \n help people lose  stocks… ',
 'RealMMOShow': 'cortez masto d \n\nq what can you do about oil companies driving up prices while making record profits \n\na that… ',
 'GregDaco': 'fed chair powell semiannual monetary policy report to congress\n\n⃣ understand hardship high inflation is causin

In [54]:
# Apply round 2 of data cleaning
import emoji

def clean_text_round2(text):
    
    text = emoji.get_emoji_regexp().sub(r'', text)
    text = text.replace("\n", "")
    
    return text

data_clean = {}

for user, tweet in data_clean_1.items():
    data_clean[user] = [clean_text_round2(tweet)]
    
data_clean

/var/folders/ly/b0d5ymx54rd755r8d506sd000000gn/T/ipykernel_31397/1311328516.py:6: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  text = emoji.get_emoji_regexp().sub(r'', text)


{'KitcoNewsNOW': ['powell on china i don’t expect we’ve seen the full effects of china lockdowns … as long as the zerocovid policy… '],
 'xglobalmarkets': ['effective fed funds rate  june  vs  june xglobalmarkets rates inflation fed economy'],
 'KVanderschrick': ['feds powell  theres some evidence of wages flattening out we are seeing a slowing in housingrate rises shoul… '],
 'pleasantville73': ['financialjuice ‘we don’t think” operative words and probably literal fed'],
 'aLtErNaTePeTeR': ['powell refused to answer a question from a senator as to what congress can do to make the feds job of fighting… '],
 'VanessaBrownTV': ['my nonexpert take on powell’s fed plan—  wreck the economy… to fix the economy  help people lose  stocks… '],
 'RealMMOShow': ['cortez masto d q what can you do about oil companies driving up prices while making record profits a that… '],
 'GregDaco': ['fed chair powell semiannual monetary policy report to congress⃣ understand hardship high inflation is causin… '

In [55]:
data_df = pd.DataFrame.from_dict(data_clean).transpose()
data_df.columns = ['tweets']
data_df = data_df.sort_index()
data_df # clean raw corpus

,tweets
BlackOutWhale,kennedy absolutly grilled powellduh fed crypt...
CurrencyGuy,powell price stability is really the bedrock o...
DennisKelleher,federalreserve chair powell asked senatebankin...
EconomicsRisk,with high inflation shattering consumer and in...
FXStreetNews,powell speech focused on part of inflation we ...
...,...
vinniepalma,again a pop in crude hour following powell co...
wallstmaster,inflation was high certainly before the war in...
x2da4,powell testimony summarysenators hoping to pin...
xglobalmarkets,effective fed funds rate june vs june xglob...


In [57]:
data_df.to_pickle('clean_corpus.pkl')

In [ ]:
# Create Document-term matrix

In [61]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data_df.tweets)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_df.index
data_dtm

/Users/timal/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,absolutly,accelerate,accountn,action,address,adjustedfed,admitted,althou,amp,answer,...,words,work,world,worry,wreck,xglobalmarkets,xlf,yields,zero,zerocovid
BlackOutWhale,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CurrencyGuy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DennisKelleher,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
EconomicsRisk,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
FXStreetNews,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
vinniepalma,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
wallstmaster,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
x2da4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
xglobalmarkets,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [62]:
data_dtm.to_pickle('FED_dtm.pkl')

In [64]:
pickle.dump(cv, open('cv_stop.pkl', "wb"))